In [6]:
# Imports

from dataclasses import dataclass

import cadquery as cq

# from jupyter_cadquery import show
from ocp_vscode import show
# from tools import show

In [7]:
# Dims


@dataclass
class MeasuredDims:
    tap_pipe_diameter: float = 44.7
    tap_to_sink_edge_distance: float = 33.1
    tap_control_to_slab_dist = 40.5
    tap_control_diameter = 50.0


md = MeasuredDims()


@dataclass
class DripDims:
    # The thickness of the "floor" of the dripper
    base_thickness: float = 0.6

    # The distance the dripper should extend over the edge of the sink
    extrusion_depth: float = 5

    length: float = 180  # x axis length
    depth: float = 190  # y axis length
    wall_height: float = 3
    wall_thickness: float = 0.8

    pipe_envelope_height: float = 12
    angle: float = 3  # degs

    tap_pip_features_center_y_axis = (
        extrusion_depth + md.tap_to_sink_edge_distance + md.tap_pipe_diameter / 2
    )


@dataclass
class AuxDripperDims:
    width: float = 80


In [8]:
def make_dripper(width: float) -> cq.Workplane:
    d = DripDims()

    base = (
        cq.Workplane("XY")
        .rect(width, d.depth + d.extrusion_depth, centered=False)
        .extrude(d.base_thickness + d.wall_height)
    )

    main_body_hole = (
        cq.Workplane("XY")
        .rect(width - 2 * d.wall_thickness, d.depth - d.wall_thickness, centered=False)
        .extrude(d.wall_height)
        .translate((d.wall_thickness, d.extrusion_depth, d.base_thickness))
    )

    extrusion_hole = (
        cq.Workplane("XY")
        .rect(width, d.extrusion_depth, centered=False)
        .extrude(d.wall_height)
        .translate((0, 0, d.base_thickness))
    )

    return base - main_body_hole - extrusion_hole


def make_pipe_envelope() -> cq.Workplane:
    d = DripDims()
    md = MeasuredDims()

    return (
        cq.Workplane("XY")
        .moveTo(0, d.tap_pip_features_center_y_axis)
        .circle((md.tap_pipe_diameter / 2) + d.wall_thickness)
        .extrude(d.pipe_envelope_height)
        .rotate(
            (-1, md.tap_to_sink_edge_distance + d.extrusion_depth, 0),
            (1, md.tap_to_sink_edge_distance + d.extrusion_depth, 0),
            -d.angle,
        )
        .translate((0, 0, -2))
    )


def make_pipe_hole() -> cq.Workplane:
    d = DripDims()
    md = MeasuredDims()

    return (
        cq.Workplane("XY")
        .moveTo(0, d.tap_pip_features_center_y_axis)
        .circle((md.tap_pipe_diameter / 2))
        .extrude(d.pipe_envelope_height * 2)
        .rotate(
            (-1, md.tap_to_sink_edge_distance + d.wall_thickness + d.extrusion_depth, 0),
            (1, md.tap_to_sink_edge_distance + d.wall_thickness + d.extrusion_depth, 0),
            -d.angle,
        )
        .translate((0, 0, -5))
    )


def make_main_dripper() -> cq.Workplane:
    d = DripDims()
    md = MeasuredDims()
    dripper = make_dripper(d.length)
    pipe_envelope = make_pipe_envelope()
    pipe_hole = make_pipe_hole()

    left_edge_cut_off = (
        cq.Workplane("XY").rect(100, 100, centered=False).extrude(100).translate((-100, 0, 0))
    )
    base_cut_off = (
        cq.Workplane("XY").rect(500, 500, centered=False).extrude(100).translate((-100, 0, -100))
    )

    all = dripper + pipe_envelope - pipe_hole - left_edge_cut_off - base_cut_off
    return all


main_dripper = make_main_dripper()
_ = main_dripper.export("main_dripper.stl")
# _ = show(main_dripper)

In [9]:
def make_auxiliary_dripper():
    d = AuxDripperDims()
    dripper = make_dripper(d.width)
    pipe_envelope = make_pipe_envelope().translate((d.width, 0, 0))
    pipe_hole = make_pipe_hole().translate((d.width, 0, 0))
    right_edge_cut_off = (
        cq.Workplane("XY").rect(100, 100, centered=False).extrude(100).translate((d.width, 0, 0))
    )

    base_cut_off = (
        cq.Workplane("XY").rect(500, 500, centered=False).extrude(100).translate((-100, 0, -100))
    )

    all = dripper + pipe_envelope - pipe_hole - right_edge_cut_off - base_cut_off

    return all


aux_dripper = make_auxiliary_dripper()
_ = aux_dripper.export("aux_dripper.stl")
# _ = show(aux_dripper)

In [10]:
def make_assembly():
    d = DripDims()
    md = MeasuredDims()
    ad = AuxDripperDims()
    primary = make_main_dripper()
    aux = make_auxiliary_dripper()

    slab_thickness = 50.0
    slab_w = 400.0
    slab_d = 300.0

    slab = cq.Workplane("XY").rect(slab_w, slab_d, centered=False).extrude(slab_thickness)
    tap_pipe = (
        cq.Workplane("XY")
        # .moveTo(100, md.tap_to_sink_edge_distance + md.tap_pipe_diameter / 2)
        .circle(md.tap_pipe_diameter / 2)
        .extrude(80)
    )

    tap_ctrl_pip = cq.Workplane("YZ").circle(md.tap_control_diameter / 2).extrude(80)
    ass = cq.Assembly()
    ass.add(slab, color=cq.Color("white"))
    ass.add(
        tap_pipe,
        color=cq.Color("gray"),
        loc=cq.Location(
            (slab_w / 2, md.tap_to_sink_edge_distance + md.tap_pipe_diameter / 2, slab_thickness)
        ),
    )
    ass.add(
        tap_ctrl_pip,
        color=cq.Color("gray"),
        loc=cq.Location(
            (
                slab_w / 2 - md.tap_pipe_diameter / 2 - 20,
                md.tap_to_sink_edge_distance + md.tap_control_diameter / 2,
                slab_thickness + md.tap_control_to_slab_dist,
            )
        ),
    )

    primary = primary.rotate((-1, d.extrusion_depth, 0), (1, d.extrusion_depth, 0), d.angle)
    ass.add(
        primary,
        loc=cq.Location((slab_w / 2, -d.extrusion_depth, slab_thickness)),
    )

    aux = make_auxiliary_dripper().rotate(
        (-1, d.extrusion_depth, 0), (1, d.extrusion_depth, 0), d.angle
    )
    ass.add(
        aux,
        loc=cq.Location((slab_w / 2 - ad.width, -d.extrusion_depth, slab_thickness)),
    )

    return ass


_ = show(make_assembly())

ccccc
